In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import os
import s3fs
import matplotlib.pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.regularizers import l2

from modules.utils import pre_process_data, encoded_categorical_features

In [2]:
X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")

/var/folders/41/lv7xy5r91x7ckkgx5zqmgpxh0000gn/T/ipykernel_6485/615376286.py:1: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")


In [ ]:
X_test_copy = X_test


In [3]:
X_test["piezo_groundwater_level_category"] = 0
x_test_without_nan, y_train = pre_process_data(X_test)

/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

In [4]:
x_test_without_nan["insee_%_agri"] = x_test_without_nan["insee_%_agri"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_med_living_level"] = x_test_without_nan["insee_med_living_level"].replace(
    {'N/A - résultat non disponible': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_%_ind"] = x_test_without_nan["insee_%_ind"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_%_const"] = x_test_without_nan["insee_%_const"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)

In [5]:
X_final = encoded_categorical_features(x_test_without_nan)

In [6]:
X_final = pd.DataFrame(X_final)
X_final = X_final.fillna(X_final.mean())

In [ ]:
import xgboost as xgb

# Create an XGBClassifier instance
loaded_model = xgb.XGBClassifier()

# Load the model from a file
loaded_model.load_model("models/xgboost_model_2.json")

In [7]:
from tensorflow.keras.models import load_model

model = load_model('models/ML_model_2.keras')

In [ ]:
print(X_final.shape)


In [8]:
y_pred = model.predict(X_final)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_10" is incompatible with the layer: expected axis -1 of input shape to have value 87, but received input with shape (32, 86)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 86), dtype=float32)
  • training=False
  • mask=None

In [ ]:
y_pred = pd.DataFrame(y_pred)
y_pred.head()

In [ ]:
# Convert predictions to DataFrame


# Add a row index starting from 2331796
y_pred["row_index"] = X_test_copy["row_index"]
# Define category mapping
category_mapping = {
    4: "Very Low",
    2: "Low",
    0: "Average",
    1: "High",
    3: "Very High"
}

# Map the category values
y_pred["piezo_groundwater_level_category"] = y_pred[0].map(category_mapping)

# Display the final DataFrame
y_pred = y_pred.drop(columns=[0])

In [ ]:
X_final.info()

In [ ]:
y_pred.info()

In [ ]:
y_pred.to_csv("y_test_2.csv", index=False)